In [1]:
%load_ext autoreload

import torch
import numpy as np
from cmr.estimation import estimation

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

## Generate some data

In [2]:
def generate_data(n_sample):
    e = np.random.normal(loc=0, scale=1.0, size=[n_sample, 1])
    gamma = np.random.normal(loc=0, scale=0.1, size=[n_sample, 1])
    delta = np.random.normal(loc=0, scale=0.1, size=[n_sample, 1])

    z = np.random.uniform(low=-3, high=3, size=[n_sample, 1])
    t = np.reshape(z[:, 0], [-1, 1]) + e + gamma
    y = np.abs(t) + e + delta
    return {'t': t, 'y': y, 'z': z}

train_data = generate_data(n_sample=100)
validation_data = generate_data(n_sample=100)
test_data = generate_data(n_sample=10000)

## Define a PyTorch model $f$ and a moment function $\psi$

In [3]:
model = torch.nn.Sequential(
            torch.nn.Linear(1, 20),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(20, 3),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(3, 1)
        )

def moment_function(model_evaluation, y):
    return model_evaluation - y

## Train the model

In [4]:
trained_model, stats = estimation(model=model,  # Use any PyTorch model
                                  train_data=train_data,    # Format {'t': t, 'y': y, 'z': z}
                                  moment_function=moment_function,  # moment_function(model_eval, y) -> (n_sample, dim_y)
                                  estimation_method='DeepIV',   # Method in ['OLS', 'GMM', 'GEL', 'KernelEL', 'KernelMMR', 'SMD', 'KernelVMM', 'NeuralVMM', 'KernelELKernel', 'KernelELNeural', 'KernelFGEL', 'NeuralFGEL']
                                  estimator_kwargs=None,    # Non-default arguments for estimators (default at `kel.default_config.py`)
                                  hyperparams=None,     # Non-default hyperparams for estimators as {name: [val1, ..]}
                                  validation_data=None,     # Format {'t': t, 'y': y, 'z': z}
                                  val_loss_func=None,   # Custom validation loss: val_loss_func(model, validation_data) -> float
                                  verbose=True)
# Make prediction
y_pred = trained_model(torch.Tensor(test_data['t']))

AttributeError: module 'tensorflow.compat.v2.__internal__' has no attribute 'dispatch'